<a href="https://colab.research.google.com/github/mgencler/qmem/blob/main/qbellek_temiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y qiskit qiskit-aer plotnine matplotlib

Found existing installation: qiskit 1.3.2
Uninstalling qiskit-1.3.2:
  Successfully uninstalled qiskit-1.3.2
Found existing installation: qiskit-aer 0.16.0
Uninstalling qiskit-aer-0.16.0:
  Successfully uninstalled qiskit-aer-0.16.0
Found existing installation: plotnine 0.14.5
Uninstalling plotnine-0.14.5:
  Successfully uninstalled plotnine-0.14.5
Found existing installation: matplotlib 3.7.1
Uninstalling matplotlib-3.7.1:
  Successfully uninstalled matplotlib-3.7.1


In [ ]:
# 1. Gerekli Kütüphanelerin Yüklenmesi (Sürümlerle Birlikte)
!pip install "qiskit==1.3.2" "qiskit-aer==0.16.0" "matplotlib==3.8.0" "scipy==1.11.4" --quiet

# 2. Kütüphanelerin Import Edilmesi
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector, Operator, Pauli
from qiskit_aer import AerSimulator  # Aer 0.16.0 için yeni import
import numpy as np
import scipy.linalg
from scipy.optimize import minimize, basinhopping
import matplotlib.pyplot as plt

# 3. HSC ve FEO Kapıları (Qiskit 1.3.2 Uyumlu)
def create_HSC_gate_improved(theta):
    # Pauli matrislerini manuel oluşturma
    pauli_x = Pauli('X').to_matrix()
    pauli_y = Pauli('Y').to_matrix()
    pauli_z = Pauli('Z').to_matrix()
    pauli_xy = np.kron(pauli_x, pauli_y)
    pauli_zI = np.kron(pauli_z, np.eye(2))
    H = pauli_xy + pauli_zI
    return scipy.linalg.expm(-1j * theta * H)

def create_FEO_gate(phi):
    pauli_x = Pauli('X').to_matrix()
    pauli_z = Pauli('Z').to_matrix()
    pauli_xz = np.kron(pauli_x, pauli_z)
    I = np.eye(4)
    return np.cos(phi)*I + 1j*np.sin(phi)*pauli_xz

# 4. NuBell-α Devresi
def create_NuBell_alpha(theta, phi):
    qc = QuantumCircuit(2, 2) # İki klasik bit ekledik.
    qc.h(0)
    qc.cx(0, 1)
    qc.append(Operator(create_FEO_gate(phi)), [0,1])  # Yeni Operator() wrapper
    qc.append(Operator(create_HSC_gate_improved(theta)), [0,1])
    qc.measure([0, 1], [0, 1]) # Ölçüm Eklendi.
    return qc

# 5. TEM Hesaplama (Aer 0.16.0 Uyumlu)
def calculate_TEM(qc, theta, phi, shots=2048, topo_weight=0.7, entropy_weight=0.3):
    # Ölçümsüz ideal devreyi oluştur
    qc_ideal = QuantumCircuit(2)
    qc_ideal.h(0)
    qc_ideal.cx(0, 1)
    qc_ideal.append(Operator(create_FEO_gate(phi)), [0,1])
    qc_ideal.append(Operator(create_HSC_gate_improved(theta)), [0,1])

    # İdeal durum
    ideal_sv = Statevector(qc_ideal)
    rho_ideal = np.outer(ideal_sv, ideal_sv.conjugate())

    # Simülasyon
    simulator = AerSimulator()  # Yeni AerSimulator
    transpiled_qc = transpile(qc, simulator)
    result = simulator.run(transpiled_qc, shots=shots).result()
    counts = result.get_counts()

    # Olasılık dağılımı
    probs = np.array([counts.get(state, 0)/shots for state in ['00','01','10','11']])

    # Sıfır olasılık sorunu için daha iyi bir yaklaşım:
    num_states = len(probs)
    probs_adjusted = probs + (1/num_states) / num_states # Tüm olasılıklara paylaştırarak

    rho_real = np.diag(probs_adjusted)  # Sıfır olasılık hatasını önleme

    # TEM
    d_topo = 0.5 * np.linalg.norm(rho_ideal - rho_real, 'fro')
    delta_S = -np.sum(probs_adjusted * np.log2(probs_adjusted))
    return topo_weight*d_topo + entropy_weight*delta_S

# 6. Optimizasyon
def objective(params, shots=2048, topo_weight=0.7, entropy_weight=0.3):
    theta, phi = params
    theta = theta % (2 * np.pi) # Parametre normalizasyonu (modulo işlemi)
    phi = phi % (2 * np.pi)
    return calculate_TEM(create_NuBell_alpha(theta, phi), theta, phi, shots, topo_weight, entropy_weight)

# Global Optimizasyon İçin Basinhopping'e Hazırlık
def basinhopping_objective(params, shots=2048, topo_weight=0.7, entropy_weight=0.3):
    return objective(params, shots, topo_weight, entropy_weight)

# Parametre Tarama Fonksiyonu
def parameter_scan(initial_theta, initial_phi, shots=2048, topo_weight=0.7, entropy_weight=0.3, range_theta = 0.1, range_phi = 0.1, step = 0.02):
    best_tem = float('inf')
    best_theta = 0
    best_phi = 0

    theta_vals = np.arange(initial_theta-range_theta, initial_theta+range_theta, step)
    phi_vals = np.arange(initial_phi-range_phi, initial_phi+range_phi, step)

    for theta in theta_vals:
        for phi in phi_vals:
             tem = calculate_TEM(create_NuBell_alpha(theta, phi), theta, phi, shots, topo_weight, entropy_weight)
             if tem < best_tem:
                 best_tem = tem
                 best_theta = theta
                 best_phi = phi
    return best_theta, best_phi, best_tem

# Hibrit Optimizasyon Fonksiyonu
def hybrid_optimization(initial_theta, initial_phi, topo_weight, entropy_weight, low_shots=2048, high_shots=8192):
    # Nelder-Mead ile keşif
    result_nm = minimize(objective, [initial_theta, initial_phi], method='Nelder-Mead',
                         args=(low_shots, topo_weight, entropy_weight))

    # BFGS ile derinleştirme
    result_bfgs = minimize(objective, result_nm.x, method='BFGS',
                         args=(high_shots, topo_weight, entropy_weight))

    return result_bfgs

# Optimizasyonu Başlat
algorithms = ['COBYLA', 'Nelder-Mead', 'BFGS', 'Basin-Hopping', 'Hybrid'] # Hibrit algoritma eklendi
shots_values = [2048]
tem_weights = [(0.7, 0.3), (0.5, 0.5), (0.3, 0.7)]

for algo in algorithms:
    for shots in shots_values:
        for topo_weight, entropy_weight in tem_weights:
          print(f"\n--- Optimizasyon Algoritması: {algo}, Shots: {shots}, topo_weight:{topo_weight}, entropy_weight:{entropy_weight} ---")
          best_result = None
          best_tem = float('inf')
          for _ in range(5):  # Optimizasyonu birkaç kez tekrarla
              initial_theta = np.random.uniform(0, np.pi)
              initial_phi = np.random.uniform(0, np.pi)

              if algo == 'Basin-Hopping':
                  minimizer_kwargs = {"method": "Nelder-Mead",
                                      "args" : (shots, topo_weight, entropy_weight)}
                  result = basinhopping(basinhopping_objective, [initial_theta, initial_phi],
                                       minimizer_kwargs=minimizer_kwargs, niter=5)
              elif algo == 'Hybrid':
                  result = hybrid_optimization(initial_theta, initial_phi, topo_weight, entropy_weight)
              else:
                  result = minimize(objective, [initial_theta, initial_phi], method=algo,
                                     args = (shots, topo_weight, entropy_weight))

              if result.fun < best_tem:
                  best_result = result
                  best_tem = result.fun

          print(f"Optimal θ = {best_result.x[0]:.3f}, φ = {best_result.x[1]:.3f}")
          print(f"Minimum TEM = {best_result.fun:.3f}")

          # Parametre Tarama (En İyi Sonuç Etrafında)
          scan_theta, scan_phi, scan_tem = parameter_scan(best_result.x[0], best_result.x[1], shots, topo_weight, entropy_weight)
          print(f"Parametre Taraması (En İyi Sonuç Etrafında):")
          print(f"  Optimal θ = {scan_theta:.3f}, φ = {scan_phi:.3f}, Minimum TEM = {scan_tem:.3f}")


--- Optimizasyon Algoritması: COBYLA, Shots: 2048, topo_weight:0.7, entropy_weight:0.3 ---
